In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import shutil

plt.rcParams["figure.figsize"] = (10,10)
print("Current working directory = ",os.getcwd())

In [ ]:
base_dir = 'GDXray/Castings'
list_base_dir = sorted(os.listdir(base_dir))[1:]

print('Folders in base directory = ',list_base_dir)

In [ ]:
# find folders with ground truth bounding box data (also miss out: C0002, C0029, C0034, C0041)

gt_dir = []
exclude_folders = ['C0002', 'C0029', 'C0034', 'C0041']

for folder in list_base_dir:
    pth = os.path.join(base_dir,folder)
    files = sorted(os.listdir(pth))
    if 'ground_truth.txt' in files and folder not in exclude_folders:
        gt_dir.append(folder)
    else:
        continue

print("Folders in base directory with ground truth = ",gt_dir)
print("Number of folders with ground truth = ",len(gt_dir))

In [ ]:
# create train and test split

train_folder = ['C0001', 'C0030', 'C0045', 'C0047', 'C0054', 'C0055', 'C0057', 'C0062',
          'C0008', 'C0010', 'C0015', 'C0019', 'C0035', 'C0038', 'C0039', 'C0040', 'C0060',
          'C0024', 'C0031', 'C0033', 'C0043']
          
test_folder = ['C0021', 'C0051', 'C0065',
         'C0007', 'C0032', 'C0042',
         'C0026', 'C0036']

In [ ]:
# place all images into one folder

count_images = 0

for folder in gt_dir:
    pth = os.path.join(base_dir,folder)
    files = sorted(os.listdir(pth))
    for file in files:
        if file[-3:] == 'png':
            shutil.copy(os.path.join(pth,file),os.path.join(base_dir,'Castings_images',str(count_images).zfill(4)+'.png'))
            count_images += 1
        else:
            continue


In [ ]:
# convert to yolo format and write to seperate txt files

count_gt = 0

for folder in gt_dir:
    pth = os.path.join(base_dir,folder)
    files = sorted(os.listdir(pth))
    num_in_folder = 0
    for file in files:
        if file[-3:] == 'png':
            num_in_folder += 1
    ground_truth = open(os.path.join(pth,'ground_truth.txt'),'r')
    lines = ground_truth.readlines()
    for i in range(num_in_folder):
        with open('GDXray/Castings/Castings_yolo_labels/{}.txt'.format(str(int(i+count_gt)).zfill(4)), 'w') as f:
                  pass
    for line in lines:
        bbox = line[3:-1].split('   ')
        image = plt.imread(os.path.join(base_dir,'Castings_images',str(int(float(bbox[0])-1+count_gt)).zfill(4)+'.png'))
        width = image.shape[1]
        height = image.shape[0]
        x1,x2,y1,y2 = float(bbox[1]), float(bbox[2]), float(bbox[3]), float(bbox[4])
        X, Y, W, H = format(((x1+x2)/width)/2,'.6f'), format(((y1+y2)/height)/2,'.6f'), format(((x2-x1)/width),'.6f'), format(((y2-y1)/height),'.6f')
        with open('GDXray/Castings/Castings_yolo_labels/{}.txt'.format(str(int(float(bbox[0])-1+count_gt)).zfill(4)), 'a') as f:
            f.write(f'{0} {X} {Y} {W} {H}\n')
    count_gt += num_in_folder
    #print("count = ",count_gt)
    
    

In [ ]:
# plot random image and bounding box by converting yolo

new_image_dir = os.path.join(base_dir,'Castings_images')
new_label_dir = os.path.join(base_dir,'Castings_yolo_labels')

num = 100
image_num = str(num).zfill(4)
img = plt.imread(os.path.join(new_image_dir,'{}.png'.format(image_num)))
print("Image shape = ",img.shape)
width_im = img.shape[1]
height_im = img.shape[0]

with open(os.path.join(new_label_dir,'{}.txt'.format(image_num)),'r') as f:
    lines = f.readlines()

#x_min, x_max, y_min, y_max = [],[],[],[]
#width, height = [],[]
#rects = []
fig,ax = plt.subplots()

for line in lines:
    line = line.split(' ')
    width = float(line[3])
    height = float(line[4])
    x_min = (float(line[1]) - 0.5*width)*width_im
    x_max = (float(line[1]) + 0.5*width)*width_im
    y_min = (float(line[2]) - 0.5*height)*height_im
    y_max = (float(line[2]) + 0.5*height)*height_im
    rect = patches.Rectangle((x_min,y_min), width*width_im, height*height_im, linewidth=3, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

ax.imshow(img,cmap='gray')
#ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

plt.show()

In [ ]:
# place all bboxes on images and save individually

new_image_dir = os.path.join(base_dir,'Castings_images')
new_label_dir = os.path.join(base_dir,'Castings_yolo_labels')
bbox_image_dir = os.path.join(base_dir,'Castings_images_with_bbox')
num_images = len(os.listdir(new_image_dir)[1:])

for i in range(num_images):
    num = i
    image_num = str(num).zfill(4)
    img = plt.imread(os.path.join(new_image_dir,'{}.png'.format(image_num)))
    #print("Image shape = ",img.shape)
    width_im = img.shape[1]
    height_im = img.shape[0]

    with open(os.path.join(new_label_dir,'{}.txt'.format(image_num)),'r') as f:
        lines = f.readlines()
        
    fig,ax = plt.subplots()

    for line in lines:
        line = line.split(' ')
        width = float(line[3])
        height = float(line[4])
        x_min = (float(line[1]) - 0.5*width)*width_im
        x_max = (float(line[1]) + 0.5*width)*width_im
        y_min = (float(line[2]) - 0.5*height)*height_im
        y_max = (float(line[2]) + 0.5*height)*height_im
        rect = patches.Rectangle((x_min,y_min), width*width_im, height*height_im, linewidth=3, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

#     ax.imshow(img,cmap='gray')
#     ax = plt.gca()
#     ax.axes.xaxis.set_visible(False)
#     ax.axes.yaxis.set_visible(False)
#     plt.savefig(os.path.join(bbox_image_dir,'{}.png'.format(image_num)),bbox_inches='tight')
#     plt.show()